# Tabular Playground Series (June 2021)
- This notebook covers my code for the Tabular Playground Series - June challenge, based on Random Forest classifier
- As the dataset is severely imbalanced, I have used an over-sampling technique in order to balance the dataset, particularly SMOTE (Synthetic Minority Oversampling Technique).
- In order to increase the accuracy of the predictions, I have used PCA in order to reduce the dimensions, such that the data continues to contain 99% of it's variance.
- Also, I have standardized the data, in order to remove the effect of the scales on the data.
- If you have any queries, feel free to post them in the comments section.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Importing the required libraries and modules

In [ ]:
from collections import Counter
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, PowerTransformer, MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss

## Describing the data

In [ ]:
df = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/train.csv')
df.info()

In [ ]:
# We can see that all the features follow different scales
df.describe()

In [ ]:
target = df['target']
df = df.drop(columns=['id', 'target'])
print(df.shape, len(target), type(target))

In [ ]:
target.value_counts()

### Conclusions:
- As can be seen from the above values, the dataset is highly imbalanced towards Classes 6 and 8.
- In order to balance this dataset, we will be using **SMOTE (Synthetic Minority Oversampling TEchnique)**

## SMOTE 

In [ ]:
pip install -U imbalanced-learn

In [ ]:
from imblearn.over_sampling import SMOTE
X_sam, y_sam = SMOTE().fit_resample(df, target)
print(X_sam.shape, y_sam.shape)
y_sam.value_counts()

## Pre-processing the data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_sam, y_sam, random_state=42, test_size=0.05)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
sc = StandardScaler()
sc.fit_transform(X_train)
sc.transform(X_test)

## Feature Importance

In [ ]:
pca = PCA(n_components=0.99, svd_solver='full')
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)
print(pca.n_components_)
print(X_train.shape, X_test.shape)

## Modelling the Data

In [ ]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred_prob = clf.predict_proba(X_test)
y_pred = clf.predict(X_test)

In [ ]:
print(y_pred_prob[5], y_pred)

In [ ]:
log_loss(y_test, y_pred_prob)

## Submission

In [ ]:
test_df = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/test.csv')
ids = test_df['id']
test_df = test_df.drop(columns=['id'])
sc.transform(test_df)
test_df = pca.transform(test_df)
print(test_df.shape)

In [ ]:
y_sub = clf.predict_proba(test_df)
y_sub.shape

In [ ]:
col_names = ['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6', 
             'Class_7', 'Class_8', 'Class_9']
df_sub = pd.DataFrame(data=y_sub, columns=col_names)
df_sub.shape

In [ ]:
df_sub.head()

In [ ]:
df_ids = pd.DataFrame(data=ids, columns=['id'])
print(df_ids.shape)
df_ids.head()

In [ ]:
df_final = pd.concat([df_ids, df_sub], axis=1)
df_final.head()

In [ ]:
df_final.to_csv('submission.csv', index=False)